In [78]:
#! pip3 install beautifulsoup4

import requests
import pandas as pd
from bs4 import BeautifulSoup

# Download match data in HTML format 

We will loop through all pages containing result data for league we are interested in. For each page we will find the result daa which ae contained wihtin the class `row_fixutre_row` and an `<a>` tag. We will save each row;s HTML to a list

In [41]:
# need to set up a user agent so we don't get a forbidden message when we try to scrape
agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}

# will store the results here
matches_html = []

# pages with results
urls = [
    "https://www.dublingaa.ie/competitions/results/136273#focus"
    "https://www.dublingaa.ie/competitions/results/136273/P100#focus"
]

for url in urls:
    # get the page
    response = requests.get(url, headers=agent)
    # parse html response for rows caontaining match data
    soup = BeautifulSoup(response.text, "html.parser")
    match_reports = soup.findAll("a", {"class": "row fixture_row"})
    
    matches_html = matches_html + match_reports
    
print("Done!")

Done!


# Parse required fields from HTML and store as dictionary 

Next we will loop through our HTML rows and pull out the required information. We will store this data in a list of disctionaries.

In [83]:
# list we will use to store parsed match results
results_data = []

# iterate through each match html
for match in matches_html:
    children = match.findChildren("div")
    
    # skip any matches which didn't take place
    if children[4].text.strip() != "W/O":
        # need to do some preprocessing on scorline text to split into goals and points
        home_team_score = children[4].text.strip().split('-')
        away_team_score = children[6].text.strip().split('-')

        # add the cleaned data to our list as a dict
        results_data.append(
            {
                'date': children[0].text.strip(),
                'time': children[1].text.strip(),
                'location': children[2].text.strip(),
                'home_team': children[3].text.strip(),
                'home_team_goals': int(home_team_score[0]),
                'home_team_points': int(home_team_score[1]),
                'away_team':children[7].text.strip(),
                'away_team_goals': int(away_team_score[0]),
                'away_team_points': int(away_team_score[1]),

            }
        )
    
print("Done!")

Done!


# Convert to dataframe and save locally as pickle 

Finally we will convert this list of dictionaries to a dataframe and save it locally as a pickle.

In [84]:
gaa_df = pd.DataFrame(results_data)
gaa_df.to_pickle("data/dublin_gaa_results.pkl")
gaa_df.head()

,date,time,location,home_team,home_team_goals,home_team_points,away_team,away_team_goals,away_team_points
0,09 Nov,14:00,Martin Savage Park,St Oliver Plunketts ER,3,13,Templeogue Synge Street,1,9
1,23 Oct,19:30,Lawless Park,Fingallians,3,19,Na Fianna,3,15
2,19 Oct,16:00,Dolphin Park,Templeogue Synge Street,1,12,Cuala,4,14
3,19 Oct,16:00,St Mobhi Road,Na Fianna,0,9,St Oliver Plunketts ER,1,12
4,05 Oct,16:00,Hyde Park,Cuala,7,11,St Maurs,1,5
